In [1]:
!nvidia-smi

Tue Jul 23 13:50:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.171.04             Driver Version: 535.171.04   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A30                     Off | 00000000:02:00.0 Off |                   On |
| N/A   29C    P0              25W / 165W |     50MiB / 24576MiB |     N/A      Default |
|                                         |                      |              Enabled |
+-----------------------------------------+----------------------+--

In [2]:
from dataclasses import dataclass
from functools import partial
from math import floor 
from typing import Callable, Tuple, Any

import jax
from jax import debug
import jax.numpy as jnp
import flax.linen as nn
import optax
from chex import ArrayTree
from qdax.core.containers.repertoire import Repertoire
from qdax.types import Descriptor, ExtraScores, Fitness, Genotype, RNGKey
from qdax.environments.base_wrappers import QDEnv
from qdax.core.neuroevolution.buffers.buffer import QDTransition, QDMCTransition
#from qdax.core.neuroevolution.buffers.trajectory_buffer import TrajectoryBuffer
import flashbax as fbx
import chex
from rein_related import *

from qdax.core.emitters.emitter import Emitter, EmitterState

In [3]:
import os

os.environ['MPLCONFIGDIR'] = '/tmp/matplotlib'
os.environ['WANDB_CACHE_DIR'] = '/tmp/wandb_cache'
os.environ['JAX_LOG_COMPILATION'] = '1'

import logging
import time
from dataclasses import dataclass
from functools import partial
from math import floor
from typing import Any, Dict, Tuple, List, Callable
import pickle
from flax import serialization
#logging.basicConfig(level=logging.DEBUG)
import hydra
from omegaconf import OmegaConf, DictConfig
import jax
import jax.numpy as jnp
from hydra.core.config_store import ConfigStore
from qdax.core.map_elites import MAPElites
from qdax.types import RNGKey, Genotype
from qdax.utils.sampling import sampling 
from qdax.core.containers.mapelites_repertoire import compute_cvt_centroids, MapElitesRepertoire
from qdax.core.neuroevolution.networks.networks import MLPMCPG
from qdax.core.emitters.me_mcpg_emitter import MEMCPGConfig, MEMCPGEmitter
#from qdax.core.emitters.rein_emitter_advanced import REINaiveConfig, REINaiveEmitter
from qdax.core.neuroevolution.buffers.buffer import QDTransition
from qdax.environments import behavior_descriptor_extractor
from qdax.tasks.brax_envs import reset_based_scoring_function_brax_envs as scoring_function
from utils import Config, get_env
from qdax.core.emitters.mutation_operators import isoline_variation
import wandb
from qdax.utils.metrics import CSVLogger, default_qd_metrics
from qdax.utils.plotting import plot_map_elites_results, plot_2d_map_elites_repertoire
import matplotlib.pyplot as plt
from set_up_brax import get_reward_offset_brax
from qdax import environments_v1, environments

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline
import jax.numpy as jnp  # Assuming you are using jax.numpy as jnp

In [5]:
def get_env(env_name):
    if env_name == "hopper_uni":
        episode_length = 1000
        
        env = environments_v1.create(env_name, episode_length=episode_length)
    elif env_name == "halfcheetah_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)
        
    elif env_name == "walker2d_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length)	
    elif env_name == "ant_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=True)
    elif env_name == "humanoid_uni":
        episode_length = 1000

        env = environments_v1.create(env_name, episode_length=episode_length, exclude_current_positions_from_observation=True)	
    '''
    elif env_name == "ant_omni":
        episode_length = 250
        max_bd = 30.

        env = environments.create(env_name, episode_length=episode_length, use_contact_forces=False, exclude_current_positions_from_observation=False)	
    elif env_name == "humanoid_uni":
        episode_length = 1000
        max_bd = 1.

        env = environments.create(env_name, episode_length=episode_length)	
    else:
        ValueError(f"Environment {env_name} not supported.")
    '''
    return env

In [6]:
@dataclass
class Config:
    """Configuration from this experiment script
    """
    # Env config
    #alg_name: str
    seed: int
    env_name: str
    episode_length: int
    policy_hidden_layer_sizes: Tuple[int, ...]   
    # ME config
    num_evaluations: int
    num_iterations: int
    no_agents: int
    num_samples: int
    fixed_init_state: bool
    discard_dead: bool
    # Emitter config
    iso_sigma: float
    line_sigma: float
    #crossover_percentage: float
    # Grid config 
    grid_shape: Tuple[int, ...]
    num_init_cvt_samples: int
    num_centroids: int
    # Log config
    log_period: int
    store_repertoire: bool
    store_repertoire_log_period: int
    
    # REINFORCE Parameters
    proportion_mutation_ga : float
    buffer_sample_batch_size : int
    buffer_add_batch_size: int
    adam_optimizer: bool
    learning_rate: float
    discount_rate: float
    #buffer_size: int
    clip_param: float
    no_epochs: int

In [7]:
no_epochs = [1, 2, 4]

envs = ["hopper_uni", "walker2d_uni", "ant_uni"]

os.makedirs("grad_steps_experiments/reps=16/lr=3e-5", exist_ok=True)


for env_ in envs:
    
    env_dir = f"grad_steps_experiments/reps=16/lr=3e-5/{env_}"
    os.makedirs(env_dir, exist_ok=True)
    
    for no_epoch in no_epochs:
        config = Config(
            seed=0,
            env_name=env_,
            episode_length=1000,
            policy_hidden_layer_sizes=[128, 128],
            num_evaluations=0,
            num_iterations=4000,
            num_samples=8,
            no_agents=512,
            fixed_init_state=False,
            discard_dead=False,
            grid_shape=[50, 50],
            num_init_cvt_samples=50000,
            num_centroids=1024,
            log_period=400,
            store_repertoire=True,
            store_repertoire_log_period=800,
            iso_sigma=0.005,
            line_sigma=0.05,
            proportion_mutation_ga=0.5,
            buffer_sample_batch_size=8,
            buffer_add_batch_size=512,
            no_epochs=no_epoch,
            #buffer_size=64000,
            adam_optimizer=True,
            learning_rate=3e-5,
            discount_rate=0.99,
            clip_param=0.2
        )

        random_key = jax.random.PRNGKey(config.seed)

        # Init environment
        env = get_env(env_)
        reset_fn = jax.jit(env.reset)

        # Compute the centroids
        centroids, random_key = compute_cvt_centroids(
            num_descriptors=env.behavior_descriptor_length,
            num_init_cvt_samples=config.num_init_cvt_samples,
            num_centroids=config.num_centroids,
            minval=0,
            maxval=1,
            random_key=random_key,
        )
        # Init policy network
        policy_layer_sizes = config.policy_hidden_layer_sizes #+ (env.action_size,)
        print(policy_layer_sizes)




        policy_network = MLPMCPG(
            hidden_layers_size=policy_layer_sizes,
            action_size=env.action_size,
            activation=jax.nn.tanh,
            hidden_init=jax.nn.initializers.orthogonal(scale=jnp.sqrt(2)),
            mean_init=jax.nn.initializers.orthogonal(scale=0.01),
        )


            
        '''
        policy_network = MLPMCPG(
            hidden_layers_size=policy_layer_sizes,
            action_size=env.action_size,
            activation=jax.nn.tanh,
            hidden_init=jax.nn.initializers.variance_scaling(scale=jnp.sqrt(2), mode='fan_in', distribution='uniform'),
            mean_init=jax.nn.initializers.variance_scaling(scale=0.02*jnp.sqrt(2), mode='fan_in', distribution='uniform'),
        )
        '''


        '''
        policy_network = MLPMCPG(
            hidden_layers_size=policy_layer_sizes,
            action_size=env.action_size,
            activation=jax.nn.tanh,
            hidden_init=jax.nn.initializers.lecun_uniform(),
            mean_init=jax.nn.initializers.lecun_uniform(),
        )
        '''



        # Init population of controllers

        # maybe consider adding two random keys for each policy
        random_key, subkey = jax.random.split(random_key)
        keys = jax.random.split(subkey, num=config.no_agents)
        #split_keys = jax.vmap(lambda k: jax.random.split(k, 2))(keys)
        #keys1, keys2 = split_keys[:, 0], split_keys[:, 1]
        fake_batch_obs = jnp.zeros(shape=(config.no_agents, env.observation_size))
        init_params = jax.vmap(policy_network.init)(keys, fake_batch_obs)

        param_count = sum(x[0].size for x in jax.tree_util.tree_leaves(init_params))
        print("Number of parameters in policy_network: ", param_count)

        # Define the fonction to play a step with the policy in the environment
        @jax.jit
        def play_step_fn(env_state, policy_params, random_key):
            #random_key, subkey = jax.random.split(random_key)
            actions, logp = policy_network.apply(policy_params, env_state.obs)
            #logp = policy_network.apply(policy_params, env_state.obs, actions, method=policy_network.logp)
            state_desc = env_state.info["state_descriptor"]
            next_state = env.step(env_state, actions)

            transition = QDMCTransition(
                obs=env_state.obs,
                next_obs=next_state.obs,
                rewards=next_state.reward,
                dones=next_state.done,
                truncations=next_state.info["truncation"],
                actions=actions,
                state_desc=state_desc,
                next_state_desc=next_state.info["state_descriptor"],
                logp=logp,
                #desc=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
                #desc_prime=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
            )

            return next_state, policy_params, random_key, transition


        # Prepare the scoring function
        bd_extraction_fn = behavior_descriptor_extractor['ant_uni']
        scoring_fn = partial(
            scoring_function,
            episode_length=env.episode_length,
            play_reset_fn=reset_fn,
            play_step_fn=play_step_fn,
            behavior_descriptor_extractor=bd_extraction_fn,
        )
        #reward_offset = get_reward_offset_brax(env, config.env_name)
        #print(f"Reward offset: {reward_offset}")

        me_scoring_fn = partial(
        sampling,
        scoring_fn=scoring_fn,
        num_samples=config.num_samples,
        )

        reward_offset = 0

        # Define a metrics function
        metrics_function = partial(
            default_qd_metrics,
            qd_offset=reward_offset * env.episode_length,
        )

        # Define the PG-emitter config

        me_mcpg_config = MEMCPGConfig(
            proportion_mutation_ga=config.proportion_mutation_ga,
            no_agents=config.no_agents,
            buffer_sample_batch_size=config.buffer_sample_batch_size,
            buffer_add_batch_size=config.buffer_add_batch_size,
            #batch_size=config.batch_size,
            #mini_batch_size=config.mini_batch_size,
            no_epochs=config.no_epochs,
            #buffer_size=config.buffer_size,
            learning_rate=config.learning_rate,
            adam_optimizer=config.adam_optimizer,
            clip_param=config.clip_param,
        )

        variation_fn = partial(
            isoline_variation, iso_sigma=config.iso_sigma, line_sigma=config.line_sigma
        )

        me_mcpg_emitter = MEMCPGEmitter(
            config=me_mcpg_config,
            policy_network=policy_network,
            env=env,
            variation_fn=variation_fn,
            )

        '''
        rein_emitter = REINaiveEmitter(
            config=rein_emitter_config,
            policy_network=policy_network,
            env=env,
            )
        '''
        '''
        me_scoring_fn = partial(
            sampling,
            scoring_fn=scoring_fn,
            num_samples=config.num_samples,
        )
        '''

        # Instantiate MAP Elites
        map_elites = MAPElites(
            scoring_function=scoring_fn,
            emitter=me_mcpg_emitter,
            metrics_function=metrics_function,
        )

        fitnesses, descriptors, extra_scores, random_key = scoring_fn(
            init_params, random_key
        )
        
        repertoire = MapElitesRepertoire.init(
            genotypes=init_params,
            fitnesses=fitnesses,
            descriptors=descriptors,
            centroids=centroids,
            extra_scores=extra_scores,
        )
        

        emitter_state, random_key = me_mcpg_emitter.init(
            random_key=random_key,
            repertoire=repertoire,
            genotypes=init_params,
            fitnesses=fitnesses,
            descriptors=descriptors,
            extra_scores=extra_scores,
        )
        
        emitter_state = me_mcpg_emitter.state_update(
            emitter_state=emitter_state,
            repertoire=repertoire,
            genotypes=init_params,
            fitnesses=fitnesses,
            descriptors=descriptors,
            extra_scores={**extra_scores}#, **extra_info},
        )
        
        emitter_state = emitter_state.emitter_states[0]
        
        


        returns = []
        old_params = init_params
        random_key = jax.random.PRNGKey(0)
        for _ in range(250):
            random_keys = jax.random.split(random_key, config.no_agents)
            new_params = me_mcpg_emitter.emitters[0].emit_mcpg(emitter_state, old_params, random_keys)
            fitnesses, descriptors, extra_scores, random_key = scoring_fn(
                new_params, random_key
            )
            emitter_state = me_mcpg_emitter.emitters[0].state_update(
                emitter_state=emitter_state,
                repertoire=repertoire,
                genotypes=new_params,
                fitnesses=fitnesses,
                descriptors=descriptors,
                extra_scores=extra_scores,
            )
            old_params = new_params
            print(f"mean fitness: {fitnesses.mean()}")
            returns.append(fitnesses)
            
        returns = jnp.array(returns)  # Assuming 'returns' is already defined as a 2D array

        # Determine the overall min and max fitness values for setting y-axis limits
        ymin = returns.min()
        ymax = returns.max()

        # Plotting
        fig, axs = plt.subplots(16, 32, figsize=(40, 80))  # Adjust the subplot grid to 16x32
        for i in range(512):  # Loop through 512 plots
            ax = axs[i // 32, i % 32]  # This assumes a 16x32 grid of subplots
            ax.plot(returns[:, i])
            ax.set_title(f"Policy {i+1}", fontsize=8)
            ax.set_xlabel('Steps', fontsize=6)
            ax.set_ylabel('Returns', fontsize=6)
            ax.set_ylim([ymin, ymax])  # Set the same y-axis limits for all subplots
            ax.tick_params(axis='both', which='major', labelsize=6)

        plt.tight_layout()
        # Save each plot in the specified directory with the correct filename
        plot_filename = f"{env_dir}/no_epoch={no_epoch}.png"
        plt.savefig(plot_filename)
        plt.close(fig)  # Close the plot to free up memory

[128, 128]
Number of parameters in policy_network:  18566


/tmp/ipykernel_3764367/2361575427.py:217: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


mean fitness: 209.39988708496094
mean fitness: 192.29270935058594
mean fitness: 170.78390502929688
mean fitness: 173.22805786132812
mean fitness: 163.5931396484375
mean fitness: 158.0996551513672
mean fitness: 157.01419067382812
mean fitness: 154.29885864257812
mean fitness: 150.0264892578125
mean fitness: 149.0068817138672
mean fitness: 145.9395751953125
mean fitness: 140.47116088867188
mean fitness: 136.84671020507812
mean fitness: 139.73709106445312
mean fitness: 140.72586059570312
mean fitness: 136.59353637695312
mean fitness: 133.23681640625
mean fitness: 132.57098388671875
mean fitness: 130.6368865966797
mean fitness: 133.15823364257812
mean fitness: 135.79501342773438
mean fitness: 137.65780639648438
mean fitness: 139.79196166992188
mean fitness: 144.31436157226562
mean fitness: 147.9417724609375
mean fitness: 150.1173553466797
mean fitness: 152.01205444335938
mean fitness: 157.0152587890625
mean fitness: 163.62692260742188
mean fitness: 170.85585021972656
mean fitness: 173.2052

/tmp/ipykernel_3764367/2361575427.py:217: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


mean fitness: 174.9788818359375
mean fitness: 165.9071044921875
mean fitness: 138.14114379882812
mean fitness: 138.21328735351562
mean fitness: 123.52970886230469
mean fitness: 127.36125183105469
mean fitness: 126.10984802246094
mean fitness: 127.22628784179688
mean fitness: 129.01895141601562
mean fitness: 134.4825897216797
mean fitness: 140.89608764648438
mean fitness: 147.59390258789062
mean fitness: 153.84872436523438
mean fitness: 162.78680419921875
mean fitness: 171.49282836914062
mean fitness: 172.2177734375
mean fitness: 187.87109375
mean fitness: 190.90696716308594
mean fitness: 196.85952758789062
mean fitness: 201.779052734375
mean fitness: 203.80032348632812
mean fitness: 204.780029296875
mean fitness: 206.43783569335938
mean fitness: 204.35885620117188
mean fitness: 204.7257843017578
mean fitness: 205.87786865234375
mean fitness: 210.52078247070312
mean fitness: 208.10984802246094
mean fitness: 206.6123046875
mean fitness: 206.81997680664062
mean fitness: 209.27069091796875

...

mean fitness: 994.985107421875
mean fitness: 995.2593994140625
mean fitness: 995.0162353515625
mean fitness: 994.2904052734375
mean fitness: 993.31396484375
mean fitness: 992.239990234375
mean fitness: 991.5517578125
mean fitness: 990.6728515625
mean fitness: 989.8463134765625
mean fitness: 989.0762329101562
mean fitness: 988.5677490234375
mean fitness: 987.7505493164062
mean fitness: 987.3770141601562
mean fitness: 987.1863403320312
mean fitness: 986.5068359375
mean fitness: 986.33447265625
mean fitness: 986.197998046875
mean fitness: 985.893798828125
mean fitness: 985.573486328125
mean fitness: 984.998291015625
mean fitness: 985.0189819335938
mean fitness: 985.2364501953125
mean fitness: 984.9347534179688
mean fitness: 984.5489501953125
mean fitness: 984.2646484375
mean fitness: 983.7521362304688
mean fitness: 983.6376342773438
mean fitness: 983.600830078125
mean fitness: 983.0612182617188
mean fitness: 983.064453125
mean fitness: 983.2916259765625
mean fitness: 983.0719604492188
mea

/tmp/ipykernel_3764367/2361575427.py:217: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


mean fitness: 993.494384765625
mean fitness: 992.096435546875
mean fitness: 989.697509765625
mean fitness: 988.055419921875
mean fitness: 987.2362060546875
mean fitness: 986.62939453125
mean fitness: 986.48974609375
mean fitness: 985.994384765625
mean fitness: 985.5452880859375
mean fitness: 985.871826171875
mean fitness: 985.1212158203125
mean fitness: 985.2131958007812
mean fitness: 985.37841796875
mean fitness: 984.87548828125
mean fitness: 984.4653930664062
mean fitness: 984.9769287109375
mean fitness: 984.808349609375
mean fitness: 984.0584106445312
mean fitness: 981.944091796875
mean fitness: 982.9229736328125
mean fitness: 982.7091064453125
mean fitness: 981.6505737304688
mean fitness: 981.0753173828125
mean fitness: 982.0671997070312
mean fitness: 982.1209716796875
mean fitness: 980.87353515625
mean fitness: 980.8208618164062
mean fitness: 979.66748046875
mean fitness: 980.6472778320312
mean fitness: 979.4567260742188
mean fitness: 979.4365234375
mean fitness: 977.0966796875
me

/tmp/ipykernel_3764367/2361575427.py:217: UserWarning: This type of repertoire does not store the extra scores computed by the scoring function
  repertoire = MapElitesRepertoire.init(


mean fitness: 982.3748779296875
mean fitness: 984.0264892578125
mean fitness: 986.14208984375
mean fitness: 985.5673828125
mean fitness: 986.7904663085938
mean fitness: 986.34716796875
mean fitness: 986.9097900390625
mean fitness: 985.55126953125
mean fitness: 985.3809814453125
mean fitness: 985.3694458007812
mean fitness: 983.9381103515625
mean fitness: 984.0394897460938
mean fitness: 981.7835693359375
mean fitness: 979.77685546875
mean fitness: 980.4027099609375
mean fitness: 977.289794921875
mean fitness: 976.527587890625
mean fitness: 973.8755493164062
mean fitness: 974.879638671875
mean fitness: 974.0469970703125
mean fitness: 972.4190673828125
mean fitness: 979.4581909179688
mean fitness: 978.135009765625
mean fitness: 978.3341064453125
mean fitness: 980.7447509765625
mean fitness: 982.9163818359375
mean fitness: 982.1976928710938
mean fitness: 984.048583984375
mean fitness: 983.876708984375
mean fitness: 985.0680541992188
mean fitness: 984.1965942382812
mean fitness: 984.7489013